### DATA 302 - Final
#### Due Monday, May 13 by 5:00pm

Submit your queries as a .data302 file.

Make sure to check your submission file using the dbga.validate_submission_format() tool! If you need a refresher, refer back to the Checkpoint: How to Submit SQL Assignments (and ask if you still have any questions).

Include in the submission notes if you used online resources or worked with others. **Some of you have been very brief in previous submissions. You need to be specific, do not just say, "I worked with Joe", or, "I used stackexchange"!** Refer to the Collaboration policy in the syllabus for further clarification.

Other tips:

- Don't try to do too much at once. Break the problems into smaller subproblems and make ample use of CTEs, subqueries, or VIEWs. Many of the mistakes in previous assignments were due to trying to do everything in a single SELECT statement.
- If you are using VIEWs, remember to add the CREATE VIEW statements into a list with the key 'setup' in your .data302 file (an example notebook is posted with this assignment). The database your code is checked against will not initially contain any VIEWs (even if we wrote them in class). Running the dbga.validate_submission_format() tool will catch this if you forget, as it first clones your database, then clears any existing views from the clone, then runs the rest of your SQL code.
- Make sure your zip codes are formatted correctly (i.e., 5 characters long)!
- Watch out for integer division. Spot check your results and do some of the math by hand.
- Write additional SQL and/or python code to double check your results!

Thanks for a great semester!

_May you do good and not evil_<br>
_May you find forgiveness for yourself and forgive others_<br>
_May you share freely, never taking more than you give._

-Richard Hipp

---
1) Generate a summary, by month and year of how our store is performing.

Have your query return the following:
 - year
 - month
 - Sales: total sales for the month (i.e., sum of qty * unit price). Don't forget to divide by 100 so the results are in dollars rather than pennies.
 - NumInvoices: number of invoice_id (not invoice lines) for the month
 - NumCust: number of _distinct_ customers who made a purchase (i.e. only count the customer at most once per month)
 - InvoicesPerCust: average number of invoices per customer (i.e. NumInvoices/NumCust)
 - SalesPerCust: average sales per customer (i.e. Sales/NumCust)
 - SalesPerInvoice: average sales per order (i.e. Sales/NumInvoices)

Order the results by year ASC, month ASC

_Hint: Watch out for integer division!_

_Double check your results by doing some of the math by hand. Also, since we have 99 months worth of data, you should have 99 records in your results_

In [1]:
import Store_SQL_Checkpoint_4_5_Apr_26 as ss
from importlib import reload
import base_db
reload(ss);

import pandas as pd


from dbga import validate_submission_format
import json
from collections import defaultdict as dd

In [2]:
my_sql = dd(dict)

In [3]:
db = ss.StoreDB(path_db='store.sqlite')

In [4]:
sql1 = """

WITH 

Sales_t as
(
    SELECT year, month, sum(qty*unit_price)/100. as Sales
    FROM tInvDetail
    JOIN tProd USING(prod_id)
    JOIN tInv USING(invoice_id)
    GROUP BY year, month
),

NumInvoices_t as 
(
    SELECT year, month, count(DISTINCT invoice_id) as NumInvoices
    FROM tInv
    GROUP BY year, month
),

NumCust_t as
(
    SELECT year, month, count(DISTINCT cust_id) as NumCust
    FROM tInv
    GROUP BY year, month
)

SELECT 
    NumInvoices_t.year,
    NumInvoices_t.month,
    Sales_t.Sales,
    NumInvoices,
    NumCust,
    NumInvoices*1.0/NumCust as InvoicesPerCust,
    Sales/NumCust as SalesPerCust,
    Sales/NumInvoices as SalesPerInvoice
    
FROM NumInvoices_t
JOIN NumCust_t ON NumInvoices_t.year = NumCust_t.year AND NumInvoices_t.month = NumCust_t.month
JOIN Sales_t ON NumInvoices_t.year = Sales_t.year AND NumInvoices_t.month = Sales_t.month
ORDER BY NumInvoices_t.year ASC, NumInvoices_t.month ASC
;"""

db.run_query(sql1)

,year,month,Sales,NumInvoices,NumCust,InvoicesPerCust,SalesPerCust,SalesPerInvoice
0,2016,01,10279.96,148,124,1.193548,82.902903,69.459189
1,2016,02,12277.69,149,126,1.182540,97.441984,82.400604
2,2016,03,14079.24,153,131,1.167939,107.475115,92.021176
3,2016,04,12915.68,181,131,1.381679,98.592977,71.357348
4,2016,05,12788.72,157,130,1.207692,98.374769,81.456815
...,...,...,...,...,...,...,...,...
94,2023,11,30216.80,363,215,1.688372,140.543256,83.241873
95,2023,12,30514.59,352,216,1.629630,141.271250,86.689176
96,2024,01,27461.98,368,216,1.703704,127.138796,74.624946
97,2024,02,30016.48,372,216,1.722222,138.965185,80.689462


In [5]:
my_sql['q1']['sql'] = sql1

---
2) For each year in the data, determine the month that had the lowest sales.

Since there are 9 years of data, this should return 9 records.

Return:
- year
- month
- sales (sum of qty * unit_price). Don't forget to divide by 100

Order by year ASC, month ASC

In [6]:
sql2 = """

WITH

MonthlySales as 
(
    SELECT year, month, sum(qty*unit_price)/100. as sales
    FROM tInvDetail
    JOIN tProd USING(prod_id)
    JOIN tInv USING(invoice_id)
    GROUP BY year, month
),

MinMonthlySalesThatYear as 
(
    SELECT year, min(sales) as min_sales
    FROM MonthlySales
    GROUP BY year
)

SELECT MonthlySales.year, MonthlySales.month, MonthlySales.sales
FROM MonthlySales
JOIN MinMonthlySalesThatYear ON MonthlySales.year = MinMonthlySalesThatYear.year AND MonthlySales.sales = MinMonthlySalesThatYear.min_sales
ORDER BY MonthlySales.year ASC, MonthlySales.month ASC
;"""

db.run_query(sql2)

,year,month,sales
0,2016,01,10279.96
1,2017,01,11721.39
2,2018,01,15585.54
3,2019,09,18942.84
4,2020,05,16398.19
5,2021,09,21865.55
6,2022,01,19068.84
7,2023,05,25203.78
8,2024,03,27121.64


In [7]:
my_sql['q2']['sql'] = sql2

---

3. Are there any products we haven't sold at least one of in each month? If so, report the product as well as each year/month when it was not sold.

Return:

- prod_id
- prod_desc
- unit_price (divide it by 100)
- year
- month

Order the results by year ASC, month ASC, unit_price DESC

In [8]:
sql3 = """

WITH 

AllProducts as 
(
    SELECT DISTINCT prod_id, prod_desc, unit_price 
    FROM tProd
),


AllMonths as 
(
    SELECT DISTINCT year, month 
    FROM tInv
),


AllCombs as 
(
    SELECT prod_id, prod_desc, unit_price/100. as unit_price, year, month
    FROM AllProducts
    CROSS JOIN AllMonths
),


ExistSalesCombs as
(
    SELECT DISTINCT tInvDetail.prod_id, year, month
    FROM tInvDetail
    JOIN tInv USING(invoice_id)
)



SELECT
    AllCombs.prod_id,
    AllCombs.prod_desc,
    AllCombs.unit_price,
    AllCombs.year,
    AllCombs.month
FROM AllCombs
LEFT JOIN ExistSalesCombs ON AllCombs.prod_id = ExistSalesCombs.prod_id AND AllCombs.year = ExistSalesCombs.year AND AllCombs.month = ExistSalesCombs.month
WHERE ExistSalesCombs.prod_id IS NULL
ORDER BY AllCombs.year ASC, AllCombs.month ASC, AllCombs.unit_price DESC

;"""

db.run_query(sql3)

,prod_id,prod_desc,unit_price,year,month
0,530,Chainsaw,499.99,2016,06
1,530,Chainsaw,499.99,2017,01


In [9]:
my_sql['q3']['sql'] = sql3

---

4. What are our top 5 selling products (in terms of total dollars sold)?

Return:
- prod_id
- prod_desc
- total qty sold
- total sales (don't forget to divide by 100)

Order by the total sales DESC

In [10]:
sql4 = """
SELECT 
    tProd.prod_id,
    tProd.prod_desc,
    sum(tInvDetail.qty) as total_qty_sold,
    sum(tInvDetail.qty*tProd.unit_price)/100. as total_sales
FROM tInvDetail
JOIN tProd USING(prod_id)
GROUP BY tProd.prod_id
ORDER BY total_sales DESC
LIMIT 5
;"""

db.run_query(sql4)

,prod_id,prod_desc,total_qty_sold,total_sales
0,529,Workbench,838,251400.0
1,528,Ladder,2665,213200.0
2,530,Chainsaw,400,199996.0
3,521,Drill,2831,195339.0
4,527,Toolbox,3069,153450.0


In [11]:
my_sql['q4']['sql'] = sql4

---

5. Marketing called again. They want another mailing list - this time they want a list of all customers who didn't purchase anything last month.

As usual, don't hardcode in the dates. It is likely that they will want this list every month moving forward.

Return:

- first
- last
- addr
- city
- state_id
- zip

Order by first ASC, last ASC

In [12]:
sql5 = """

WITH 

max_year_t as
(
    SELECT max(year) as MaxYear
    FROM tInv
),


max_year_month as
(
    SELECT tInv.year as max_year, max(month) as max_month
    FROM tInv
    WHERE max_year = (SELECT MaxYear FROM max_year_t)
),


LastMonthCust as
(
    SELECT DISTINCT cust_id, year, month
    FROM tInv
    WHERE year = (SELECT max_year FROM max_year_month) 
          AND month = (SELECT max_month FROM max_year_month)          
    ORDER BY cust_id
),


AllCust as 
(
    SELECT DISTINCT cust_id, first, last, addr, city, state_id, tCust.zip
    FROM tCust
    JOIN tZip USING(zip)
)



SELECT AllCust.first, AllCust.last, AllCust.addr, AllCust.city, AllCust.state_id, AllCust.zip
FROM AllCust
LEFT JOIN LastMonthCust USING(cust_id)
WHERE LastMonthCust.cust_id IS NULL
ORDER BY AllCust.first ASC, AllCust.last ASC
;"""

db.run_query(sql5)

,first,last,addr,city,state_id,zip
0,Bala-Tik,Ingram,1819 Pheasant Run,Moca,PR,00676
1,Bib Fortuna,Zhang,9177 West Avenue,Belen,NM,87002
2,C-3Po,Rivera,6661 Strawberry Lane,Granger,WY,82934
3,Captain Antilles,Curtis,2327 Prospect Street,Miami,FL,33127
4,Captain Antilles,Walters,8754 2nd Street,Douglassville,TX,75560
5,Clone Commander Cody,Greene,1220 Route 44,Milmine,IL,61855
6,Colonel Datoo,Jennings,2331 Jackson Avenue,Brooklyn,MD,21225
7,Darth Maul,Curtis,8779 Front Street,Weston,NE,68070
8,Dodonna,Ingram,5838 Route 1,Morris,OK,74445
9,Fode/Beed,Jimenez,7721 College Street,East Chicago,IN,46312


In [13]:
my_sql['q5']['sql'] = sql5

---

6. Who is our top customer (in terms of total dollars sold)? Get a report of all the products they've purchased from us.

Do not hardcode the customer into the query. Again, we'll want this to be reusable in the future without having to adjust the query.

Return:

- first
- last
- addr
- zip
- prod_id
- prod_desc
- total qty
- total sales (don't forget to divide by 100)

Order by total sales DESC

In [14]:
sql6 = """

WITH 

TopCust as 
(
    SELECT tInv.cust_id, tCust.first, tCust.last, tCust.addr, tCust.zip, sum(qty*unit_price)/100. as total_sales
    FROM tInvDetail
    JOIN tProd USING(prod_id)
    JOIN tInv USING(invoice_id)
    JOIN tCust USING(cust_id)
    GROUP BY tInv.cust_id
    ORDER BY total_sales DESC
    LIMIT 1
),


TopCustProducts as 
(
    SELECT
        tInv.cust_id,
        tProd.prod_id,
        tProd.prod_desc,
        sum(tInvDetail.qty) as total_qty,
        sum(tInvDetail.qty*tProd.unit_price)/100. as total_sales_of_the_prod
    FROM tInv
    JOIN TopCust USING(cust_id)
    JOIN tInvDetail USING(invoice_id)
    JOIN tProd USING(prod_id)
    WHERE tInv.cust_id = TopCust.cust_id
    GROUP BY tInv.cust_id, tProd.prod_id, tProd.prod_desc
    ORDER BY total_sales_of_the_prod DESC
)



SELECT
    TopCust.first,
    TopCust.last,
    TopCust.addr,
    TopCust.zip,
    TopCustProducts.prod_id,
    TopCustProducts.prod_desc,
    TopCustProducts.total_qty,
    TopCustProducts.total_sales_of_the_prod
FROM TopCust
JOIN TopCustProducts ON TopCust.cust_id = TopCustProducts.cust_id
ORDER BY TopCustProducts.total_sales_of_the_prod DESC
;"""

db.run_query(sql6)

,first,last,addr,zip,prod_id,prod_desc,total_qty,total_sales_of_the_prod
0,Snap,Mcbride,5524 Hawthorne Lane,52746,530,Chainsaw,34,16999.66
1,Snap,Mcbride,5524 Hawthorne Lane,52746,529,Workbench,45,13500.00
2,Snap,Mcbride,5524 Hawthorne Lane,52746,521,Drill,40,2760.00
3,Snap,Mcbride,5524 Hawthorne Lane,52746,528,Ladder,33,2640.00
4,Snap,Mcbride,5524 Hawthorne Lane,52746,527,Toolbox,34,1700.00
5,Snap,Mcbride,5524 Hawthorne Lane,52746,526,Wire,41,1517.00
6,Snap,Mcbride,5524 Hawthorne Lane,52746,525,Hatchet,39,1403.61
7,Snap,Mcbride,5524 Hawthorne Lane,52746,524,Axe,41,1147.59
8,Snap,Mcbride,5524 Hawthorne Lane,52746,519,Monkeywrench,37,1108.15
9,Snap,Mcbride,5524 Hawthorne Lane,52746,522,Hacksaw,41,819.18


In [15]:
my_sql['q6']['sql'] = sql6

---

7) Which customers have never purchased one or more of our products? Generate a list of all customer/product combinations that do not occur in the data.

Return:

- first
- last
- addr
- zip
- prod_id
- prod_desc

Order by first, last, addr, zip, prod_id - all ascending

In [22]:
sql7 = """

WITH 

AllComb as 
(
    SELECT cust_id, prod_id
    FROM tCust
    CROSS JOIN tProd
),


ExistedComb as
(
    SELECT DISTINCT cust_id, prod_id
    FROM tInv
    JOIN tInvDetail USING(invoice_id)
)



SELECT first, last, addr, zip, AllComb.prod_id, prod_desc
FROM AllComb
JOIN tCust USING(cust_id)
JOIN tProd USING(prod_id)
LEFT JOIN ExistedComb USING(cust_id, prod_id) 
WHERE ExistedComb.cust_id IS NULL
ORDER BY first ASC, last ASC, addr ASC, zip ASC, AllComb.prod_id ASC
;"""

db.run_query(sql7)

,first,last,addr,zip,prod_id,prod_desc
0,Bala-Tik,Benson,3204 Jackson Avenue,99778,529,Workbench
1,Bala-Tik,Benson,3204 Jackson Avenue,99778,530,Chainsaw
2,Bala-Tik,Elliott,9088 Valley Road,55003,504,Screw
3,Bala-Tik,Elliott,9088 Valley Road,55003,513,Hammer
4,Bala-Tik,Elliott,9088 Valley Road,55003,514,Plane
...,...,...,...,...,...,...
654,Wedge,Gordon,2566 2nd Street,33196,527,Toolbox
655,Wedge,Gordon,2566 2nd Street,33196,529,Workbench
656,Wedge,Gordon,2566 2nd Street,33196,530,Chainsaw
657,Wedge,Schmidt,8114 Valley Road,89427,530,Chainsaw


In [23]:
my_sql['q7']['sql'] = sql7

---

8) For each customer, report the total sales from their most recent invoice.

Return:

- first
- last
- addr
- zip
- total sales for their most recent invoice

(I'm omitting the date from the final output so we don't have to worry about how it's formatted)

Order the results by first, last, addr, zip - all ascending

In [18]:
sql8 = """

WITH 

LatestInvoice as 
(
    SELECT cust_id, max(year || '-' || month || '-01') as latest_date
    FROM tInv
    GROUP BY cust_id
),


CustSales as 
(
    SELECT tCust.first, tCust.last, tCust.addr, tCust.zip, tInv.year, tInv.month, sum(tInvDetail.qty*tProd.unit_price)/100. as total_sales
    FROM tCust
    JOIN tInv USING(cust_id)
    JOIN tInvDetail USING(invoice_id)
    JOIN tProd USING(prod_id)
    JOIN LatestInvoice ON tInv.cust_id = LatestInvoice.cust_id AND (tInv.year || '-' || tInv.month || '-01') = LatestInvoice.latest_date
    GROUP BY tCust.cust_id
)



SELECT first, last, addr, zip, total_sales
FROM CustSales
ORDER BY first ASC, last ASC, addr ASC, zip ASC
;"""

db.run_query(sql8)

,first,last,addr,zip,total_sales
0,Bala-Tik,Benson,3204 Jackson Avenue,99778,91.95
1,Bala-Tik,Elliott,9088 Valley Road,55003,107.19
2,Bala-Tik,Ingram,1819 Pheasant Run,00676,103.22
3,Bala-Tik,Lara,4007 Valley Road,27405,0.10
4,Bala-Tik,Miller,8714 Washington Street,30286,2.99
...,...,...,...,...,...
242,Unkar Plutt,Walker,2416 Pheasant Run,68030,48.98
243,Wedge,Elliott,7110 Strawberry Lane,03830,108.57
244,Wedge,Gordon,2566 2nd Street,33196,94.34
245,Wedge,Schmidt,8114 Valley Road,89427,198.44


In [19]:
my_sql['q8']['sql'] = sql8

---

9. (**Extra credit! 15 points**)

First determine the month in which we had our lowest sales. 

After that, determine the 5 lowest states in terms of total sales for that month. Make sure states with 0 sales are not excluded!

Finally, get a list of all customers in those states who had zero sales that month.

Make sure not to hardcode in any of the state or customer identifiers. This should be fully automated and reusable in the future without having to alter the code.

(Don't worry about whether or not the customer had ever been to the store yet in that month. This is just to see if you can construct some slightly more complicated logic!)

Return:

- first
- last
- addr
- city
- state_id
- zip

Order by first, last, addr, zip - all ascending

In [20]:
sql9 = """

WITH 

LowestSaleDate as
(
SELECT year, month
FROM (
    SELECT year, month, sum(qty*unit_price)/100. as total_sales
    FROM tInvDetail
    JOIN tProd USING(prod_id)
    JOIN tInv USING(invoice_id)
    GROUP BY year, month
    ORDER BY total_sales ASC
    LIMIT 1)
),


LowestSaleStates as
(
SELECT state_id, COALESCE(sum(tInvDetail.qty*tProd.unit_price)/100., 0) as total_sales
FROM tState
JOIN tZip USING(state_id)
JOIN tCust USING(zip)
LEFT JOIN tInv ON tCust.cust_id = tInv.cust_id
               AND tInv.year = (SELECT year FROM LowestSaleDate)
               AND tInv.month = (SELECT month FROM LowestSaleDate)
LEFT JOIN tInvDetail USING(invoice_id)
LEFT JOIN tProd USING(prod_id)
GROUP BY tState.state_id
ORDER BY total_sales ASC
LIMIT 5
),


cust_in_the_5_states as
(
    SELECT cust_id, first, last, zip, city, state_id, addr
    FROM tCust
    JOIN tZip USING(zip)
    JOIN LowestSaleStates USING(state_id)
    WHERE state_id = LowestSaleStates.state_id
)



SELECT cust_in_the_5_states.first, 
       cust_in_the_5_states.last, 
       cust_in_the_5_states.addr, 
       cust_in_the_5_states.city, 
       cust_in_the_5_states.state_id, 
       cust_in_the_5_states.zip
       
FROM cust_in_the_5_states
JOIN tCust USING(cust_id)
LEFT JOIN tInv ON cust_in_the_5_states.cust_id = tInv.cust_id 
               AND tInv.year = (SELECT year FROM LowestSaleDate)
               AND tInv.month = (SELECT month FROM LowestSaleDate)
WHERE tInv.invoice_id IS NULL
ORDER BY cust_in_the_5_states.first ASC, cust_in_the_5_states.last ASC, cust_in_the_5_states.addr ASC, cust_in_the_5_states.zip ASC
;"""

db.run_query(sql9)

,first,last,addr,city,state_id,zip
0,Bib Fortuna,Walker,6829 2nd Street,New York,NY,10177
1,Clone Commander Cody,Zhang,8578 Briarwood Court,Cedarburg,WI,53012
2,Dodonna,Burke,3580 Jefferson Court,Hollenberg,KS,66946
3,Gold Leader,Le,4069 Sycamore Street,Argonne,WI,54511
4,Gold Leader,Walker,8859 Lake Avenue,Athens,AL,35613
5,Lama Su,Doyle,4119 Jackson Avenue,Titusville,FL,32796
6,Lama Su,Rodriguez,2755 Devon Road,Fort Myers,FL,33907
7,On Medon,Curtis,4469 Valley Road,Caldwell,KS,67022
8,Orn Free Taa,Doyle,4493 Edgewood Road,Greenwood Lake,NY,10925
9,Padme,Le,5873 Briarwood Court,Selden,KS,67757


In [21]:
my_sql['q9']['sql'] = sql9

In [24]:
submission_name = 'AlisaFinal.data302'

with open(submission_name, 'w') as f:
    json.dump(my_sql, f)

In [25]:
path_db = 'store.sqlite'

validate_submission_format(path_db, 
                           submission_name, 
                           num_questions=9
                          )

Report saved to:  VALIDATION_AlisaFinal.txt
